In [ ]:
+import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
data_dir = '/content/drive/MyDrive/brain_tumor_dataset'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls '/content/drive/MyDrive/brain_tumor_dataset'

no  yes


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
data = []

for label in ['yes', 'no']:
    label_dir = os.path.join(data_dir, label)
    for image_file in os.listdir(label_dir):
        if image_file.endswith('.jpg'):
            image_path = os.path.join(label_dir, image_file)
            data.append((image_path, label))

df = pd.DataFrame(data, columns=['image', 'label'])

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
# Create data generators with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image',
    y_col='label',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'  # Binary classification
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image',
    y_col='label',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 135 validated image filenames belonging to 2 classes.
Found 34 validated image filenames belonging to 2 classes.


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

Epoch 1/10
5/5 [==============================] - 45s 9s/step - loss: 0.8151 - accuracy: 0.5185 - val_loss: 0.6490 - val_accuracy: 0.6176
Epoch 2/10
5/5 [==============================] - 2s 293ms/step - loss: 0.7164 - accuracy: 0.4815 - val_loss: 0.6594 - val_accuracy: 0.6176
Epoch 3/10
5/5 [==============================] - 2s 283ms/step - loss: 0.6791 - accuracy: 0.5704 - val_loss: 0.6853 - val_accuracy: 0.5588
Epoch 4/10
5/5 [==============================] - 1s 260ms/step - loss: 0.6909 - accuracy: 0.5407 - val_loss: 0.6851 - val_accuracy: 0.6176
Epoch 5/10
5/5 [==============================] - 1s 266ms/step - loss: 0.6842 - accuracy: 0.5556 - val_loss: 0.6770 - val_accuracy: 0.5882
Epoch 6/10
5/5 [==============================] - 2s 471ms/step - loss: 0.6754 - accuracy: 0.6963 - val_loss: 0.6773 - val_accuracy: 0.5882
Epoch 7/10
5/5 [==============================] - 2s 402ms/step - loss: 0.6766 - accuracy: 0.6370 - val_loss: 0.6608 - val_accuracy: 0.6176
Epoch 8/10
5/5 [======

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

2/2 [==============================] - 0s 18ms/step - loss: 0.6226 - accuracy: 0.5882
Test Accuracy: 58.82%


In [ ]:
# Predict using the model
predictions = model.predict(test_generator)
predicted_labels = (predictions > 0.5).astype(int)

2/2 [==============================] - 0s 10ms/step


In [ ]:
# Display the predicted labels (yes or no)
for i, label in enumerate(predicted_labels):
    if label == 1:
        print(f'Image {test_df.iloc[i]["image"]} has cancer (Yes)')
    else:
        print(f'Image {test_df.iloc[i]["image"]} does not have cancer (No)')

Image /content/drive/MyDrive/brain_tumor_dataset/no/No11.jpg does not have cancer (No)
Image /content/drive/MyDrive/brain_tumor_dataset/yes/Y245.jpg does not have cancer (No)
Image /content/drive/MyDrive/brain_tumor_dataset/no/31 no.jpg has cancer (Yes)
Image /content/drive/MyDrive/brain_tumor_dataset/yes/Y250.jpg has cancer (Yes)
Image /content/drive/MyDrive/brain_tumor_dataset/no/No18.jpg does not have cancer (No)
Image /content/drive/MyDrive/brain_tumor_dataset/no/no 100.jpg has cancer (Yes)
Image /content/drive/MyDrive/brain_tumor_dataset/no/N5.jpg has cancer (Yes)
Image /content/drive/MyDrive/brain_tumor_dataset/yes/Y35.jpg does not have cancer (No)
Image /content/drive/MyDrive/brain_tumor_dataset/no/44no.jpg has cancer (Yes)
Image /content/drive/MyDrive/brain_tumor_dataset/yes/Y252.jpg does not have cancer (No)
Image /content/drive/MyDrive/brain_tumor_dataset/yes/Y102.jpg has cancer (Yes)
Image /content/drive/MyDrive/brain_tumor_dataset/no/no 89.jpg does not have cancer (No)
Imag

In [ ]:
image_path = '/content/drive/MyDrive/brain_tumor_dataset/yes/Y1.jpg'

In [ ]:
img = image.load_img(image_path, target_size=(64, 64))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0  # Normalize the pixel values

In [ ]:
prediction = model.predict(img)

1/1 [==============================] - 0s 110ms/step


In [ ]:
predicted_class = 1 if prediction > 0.5 else 0

In [ ]:
if predicted_class == 1:
    print("The image has cancer (Yes)")
else:
    print("The image does not have cancer (No)")

The image has cancer (Yes)
